<a href="https://colab.research.google.com/github/SwaRad69/quantumdiving/blob/main/vqctitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-aer pandas numpy scikit-learn


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector


In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"swa69420","key":"1e8bfdc016371e3ea85f205c0d3bec05"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c titanic
!unzip titanic.zip


titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gender_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")
df.columns
df = df[["Survived", "Sex", "Pclass"]].dropna()
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
X = df[["Sex", "Pclass"]].values
y = df["Survived"].values







In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
def build_vqc(theta_sex, theta_class, params):
    """
    params = [α0, α1, β0, β1]
    """
    qc = QuantumCircuit(2)

    # --- Feature encoding ---
    qc.ry(theta_sex, 0)
    qc.ry(theta_class, 1)

    # --- Variational layer ---
    qc.ry(params[0], 0)
    qc.rz(params[1], 0)

    qc.ry(params[2], 1)
    qc.rz(params[3], 1)

    # --- Entanglement ---
    qc.cx(0, 1)

    return qc


In [ ]:
def encode_features(x):
    sex, pclass = x
    theta_sex = np.pi * sex
    theta_class = np.pi * (3 - pclass) / 2
    return theta_sex, theta_class


In [ ]:
def expectation_z(qc):
    state = Statevector.from_instruction(qc)
    probs = state.probabilities()

    # Z on qubit 0
    return (probs[0] + probs[1]) - (probs[2] + probs[3])


In [ ]:
def predict(x, params):
    theta_sex, theta_class = encode_features(x)
    qc = build_vqc(theta_sex, theta_class, params)
    z = expectation_z(qc)
    return (1 + z) / 2  # map [-1,1] → [0,1]


In [ ]:
def loss(params, X, y):
    eps = 1e-8
    losses = []

    for xi, yi in zip(X, y):
        p = predict(xi, params)
        p = np.clip(p, eps, 1 - eps)
        losses.append(-(yi*np.log(p) + (1-yi)*np.log(1-p)))

    return np.mean(losses)


In [ ]:
params = np.random.uniform(0, 2*np.pi, 4)

lr = 0.1
epochs = 40
delta = 0.01

for epoch in range(epochs):
    grads = np.zeros_like(params)

    for i in range(len(params)):
        p_plus = params.copy()
        p_minus = params.copy()

        p_plus[i] += delta
        p_minus[i] -= delta

        grads[i] = (
            loss(p_plus, X_train, y_train)
            - loss(p_minus, X_train, y_train)
        ) / (2 * delta)

    params -= lr * grads

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss = {loss(params, X_train, y_train):.4f}")


Epoch 0, Loss = 0.6832
Epoch 5, Loss = 0.6661
Epoch 10, Loss = 0.6544
Epoch 15, Loss = 0.6464
Epoch 20, Loss = 0.6407
Epoch 25, Loss = 0.6365
Epoch 30, Loss = 0.6332
Epoch 35, Loss = 0.6304


In [ ]:
from sklearn.metrics import accuracy_score

y_pred = []
for x in X_test:
    p = predict(x, params)
    y_pred.append(1 if p > 0.5 else 0)

acc = accuracy_score(y_test, y_pred)
print("Test accuracy:", acc)


Test accuracy: 0.6312849162011173


In [ ]:
!pip install qiskit qiskit-ibm-runtime


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token="api key",
    overwrite=True
)



service = QiskitRuntimeService(instance="vqc wala")


In [ ]:

print(service.instances())



[{'crn': 'crn:v1:bluemix:public:quantum-computing:us-east:a/89d3fbbfad7948f3aad6d9d1795463db:382da6d8-980e-4541-a89f-7e9b7f2e04c8::', 'plan': 'open', 'name': 'vqc wala', 'tags': [], 'pricing_type': 'free'}]


In [ ]:
def predict_hardware(x, params, shots=1024):
    theta_sex, theta_class = encode_features(x)
    qc = build_vqc(theta_sex, theta_class, params)

    job = estimator.run(
        circuits=[qc],
        observables=[observable],
        shots=shots
    )
    exp_z = job.result().values[0]
    return (1 + exp_z) / 2


In [ ]:
backend = service.least_busy(simulator=False, operational=True)
print("Using backend:", backend.name)


Using backend: ibm_fez


In [ ]:
from qiskit_ibm_runtime import Estimator
from qiskit.quantum_info import SparsePauliOp

estimator = Estimator(
    mode=backend,
    options={"default_shots": 256}
)

def make_z_observable(backend, physical_qubit=0):
    n = backend.num_qubits
    pauli = ["I"] * n
    pauli[n - 1 - physical_qubit] = "Z"   # Qiskit ordering
    return SparsePauliOp.from_list([("".join(pauli), 1.0)])

observable = make_z_observable(backend, physical_qubit=0)



In [ ]:
from qiskit import transpile

def predict_hardware(x, params):
    theta_sex, theta_class = encode_features(x)
    qc = build_vqc(theta_sex, theta_class, params)

    qc_t = transpile(
        qc,
        backend=backend,
        optimization_level=1
    )

    job = estimator.run(
        [(qc_t, observable)]
    )

    # ✅ Estimator V2 result access
    exp_z = job.result().pub_results[0].data.evs[0]

    return (1 + exp_z) / 2


In [85]:
from qiskit import transpile

# pick ONE test sample
x = X_test[0]
theta_sex, theta_class = encode_features(x)

# build logical circuit
qc = build_vqc(theta_sex, theta_class, params)

# transpile to hardware
qc_t = transpile(
    qc,
    backend=backend,
    optimization_level=1
)

print("Transpiled qubits:", qc_t.num_qubits)


Transpiled qubits: 156


In [91]:
from qiskit import transpile

def predict_hardware(x, params):
    theta_sex, theta_class = encode_features(x)
    qc = build_vqc(theta_sex, theta_class, params)

    qc_t = transpile(
        qc,
        backend=backend,
        optimization_level=1
    )

    job = estimator.run([(qc_t, observable)])
    res = job.result()

    # 🔑 evs is a scalar in your Runtime version
    exp_z = float(res[0].data.evs)

    return (1 + exp_z) / 2


In [92]:
p = predict_hardware(X_test[0], params)
print("Hardware prediction:", p)


Hardware prediction: 0.29687597236915797


In [87]:
job = estimator.run([(qc_t, observable)])
res = job.result()

print(res)
print(type(res))
print(dir(res))


PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 256, 'target_precision': 0.0625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 4})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})
<class 'qiskit.primitives.containers.primitive_result.PrimitiveResult'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '_

In [93]:
from sklearn.metrics import accuracy_score

N = 10

y_pred_hw = []
y_true_hw = y_test[:N]

for i in range(N):
    p = predict_hardware(X_test[i], params)
    pred_label = 1 if p > 0.5 else 0
    y_pred_hw.append(pred_label)

    print(
        f"Sample {i}: "
        f"pred_prob={p:.3f}, "
        f"pred_label={pred_label}, "
        f"true_label={y_true_hw[i]}"
    )

acc_hw = accuracy_score(y_true_hw, y_pred_hw)
print("Hardware accuracy (subset):", acc_hw)


Sample 0: pred_prob=0.301, pred_label=0, true_label=1
Sample 1: pred_prob=0.301, pred_label=0, true_label=0
Sample 2: pred_prob=0.253, pred_label=0, true_label=0
Sample 3: pred_prob=0.731, pred_label=1, true_label=1
Sample 4: pred_prob=0.680, pred_label=1, true_label=1
Sample 5: pred_prob=0.755, pred_label=1, true_label=1
Sample 6: pred_prob=0.707, pred_label=1, true_label=1
Sample 7: pred_prob=0.285, pred_label=0, true_label=0
Sample 8: pred_prob=0.735, pred_label=1, true_label=1
Sample 9: pred_prob=0.767, pred_label=1, true_label=1
Hardware accuracy (subset): 0.9
